In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt 
import pickle
import glob
import os
import sys
import plotly.express as px

In [43]:
def import_data(path):
    """Gets data from our excel template, drops top metadata"""
    data = pd.read_excel(path)
    data = data.rename(columns=data.iloc[2]).drop(data.index[:3])
    #data = data.dropna(axis=1, how='all')
    #print(data[data.index.duplicated()])
    
    return data

def reindex_data(data):
    """Reindexes all data to D format, expands dates"""
    ### THIS MIGHT HELP?? ###
    data['Date_1'] = pd.to_datetime(data['Date_1'])
    idx = pd.date_range(data['Date_1'].min(), data['Date_1'].max(), freq='D')
    
    rule_dfs = []
    reindexed_dfs = []
    start = -2
    end = len(data.columns)
    limit = (-1 * len(data.columns)) - 1

    for i in range(len(data.columns)):
        curr_df = data.iloc[:,start:end]
        rule_dfs.append(curr_df)
        start = start - 2
        end = end - 2

        if start < limit:
            break
    
    for df in rule_dfs:
        #print(df)
        df = df.set_index(df.columns[0])
        df = df.dropna()
        df = df.apply(lambda x : x.reindex(idx)).ffill().fillna(method="bfill")
        reindexed_dfs.append(df)
    
    final_df = pd.concat(reindexed_dfs, axis = 1)
    return final_df

def get_dataset(path):
    return reindex_data(import_data(path))

In [44]:
#path = "data/rule_data - Copy.xlsx"
path = "USEQ_data.xlsx"
data = get_dataset(path)
data

,Credit Sussie Fear Barometer,SPX Forward CAPE Ratio,TSY Yield 3M,SPX FWD Yield,MXUS Index Divdend Yield,MXUS Index Debt to Equity,MXUS Index ROE,MXUS Index PE,SPX Volume,Total Number of New 52 Week Lows,...,LME Copper Futures,ISM Non-Manufacturing PMI,Citi Economic Surprise Index US,Citi Economic Surprise Index Eurozone,Bloomberg Recession Probability Index,Bloomberg Commodity Metal Index,US Auto Sales,Bloomberg Eco US Surprise Index,TSY Yield 2Y,TSY Yield 10Y
2015-01-01,34.50,1.82,0.0355,45.15,2.01,108.47,14.64,18.7105,435587739.0,351.0,...,6255.0,56.2,28.3,9.0,10.0,180.15,16.66,0.059,0.666,2.172
2015-01-02,34.50,1.82,0.0152,45.15,2.01,108.43,14.64,18.7105,435587739.0,351.0,...,6255.0,56.2,28.3,9.0,10.0,180.15,16.66,0.059,0.667,2.111
2015-01-03,34.50,1.82,0.0152,45.15,2.01,108.43,14.64,18.7105,435587739.0,351.0,...,6255.0,56.2,28.3,9.0,10.0,180.15,16.66,0.059,0.667,2.111
2015-01-04,34.50,1.82,0.0152,45.15,2.01,108.43,14.64,18.7105,435587739.0,351.0,...,6255.0,56.2,28.3,9.0,10.0,180.15,16.66,0.059,0.667,2.111
2015-01-05,33.51,1.82,0.0101,45.20,2.05,108.44,14.64,18.3631,621869485.0,594.0,...,6145.0,56.2,27.6,12.4,10.0,177.78,16.66,0.025,0.659,2.033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-20,19.14,1.55,2.6451,69.92,1.51,114.39,19.92,21.2602,646495291.0,309.0,...,8078.5,56.7,-16.4,-51.4,50.0,237.88,13.35,-0.114,3.235,2.975
2022-08-21,19.14,1.55,2.6451,69.92,1.51,114.39,19.92,21.2602,646495291.0,309.0,...,8078.5,56.7,-16.4,-51.4,50.0,237.88,13.35,-0.114,3.235,2.975
2022-08-22,18.01,1.55,2.6962,69.89,1.54,114.39,19.92,20.8037,536127321.0,511.0,...,8027.0,56.7,-9.8,-49.9,50.0,238.34,13.35,-0.114,3.312,3.017
2022-08-23,17.47,1.55,2.6694,69.87,1.54,114.39,19.92,20.7598,454368791.0,387.0,...,8120.0,56.7,-21.6,-45.8,50.0,239.59,13.35,-0.131,3.302,3.048


In [45]:
data.to_csv('USEQ_raw_data.csv')

In [46]:
(
# OG Rule Class
#  class Rule:
#     def __init__(self, name, bucket, feature1, feature2, data=data, spread1=None, spread2=None):
#         self.name = name
#         self.bucket = bucket
#         self.data = data
#         self.feature1 = feature1
#         self.feature2 = feature2
#         self.spread1 = spread1
#         self.spread2 = spread2

#         #May add these in order to deal with rules that have diff numbers
#         #self.over_num = over_num
#         #self.under_num = under_num

#         #add lag by certain days funtion as well?
    
#     def generate_features(self):
#         # Check to see if spread needs to be calculated
#         if self.spread1 and self.spread2:
#             if isinstance(self.feature1, str) and "pread" in self.feature1:
#                 self.data[self.feature1] = self.data[self.spread1] - self.data[self.spread2]
#                 self.data[self.feature2] = self.data[self.spread1] - self.data[self.spread2] #added
#             #this only makes feature 2 - took out an replaced above
#             #elif isinstance(self.feature2, str) and "pread" in self.feature2:
#             #    self.data[self.feature2] = self.data[self.spread1] - self.data[self.spread2]
#             else:
#                 print("Spread Parameters inputted incorrectly!")
#                 return
        
#         # Calculate necessary data for feature1 (sma, zscore)
#         if type(self.feature1) == int or type(self.feature1) == float:
#             pass       
#         elif "zscore" in self.feature1:
#             days = int(''.join([n for n in self.feature1.split("_")[1] if n.isdigit()]))
#             self.data["{}_{}zscore".format(self.name, days)] = (
#                 (self.data[self.name].rolling(1).mean() - self.data[self.name].rolling(days).mean()) / 
#                 self.data[self.name].rolling(days).std())
#         elif "sma" in self.feature1:
#             days = int(''.join([n for n in self.feature1.split("_")[1] if n.isdigit()]))
#             #if a spread then uses its feature name
#             if self.spread1 and self.spread2:
#                 self.data[self.feature1] = self.data[self.feature1].rolling(days).mean()
#             else:
#                 self.data["{}_{}sma".format(self.name, days)] = self.data[self.name].rolling(days).mean()
#         elif "ema" in self.feature1: 
#             days = int(''.join([n for n in self.feature1.split("_")[1] if n.isdigit()]))
#             self.data["{}_{}ema".format(self.name, days)] = self.data[self.name].ewm(span=days).mean()
        
#         # Calculate necessary data for feature2 (sma, zscore)
#         if type(self.feature2) == int or type(self.feature2) == float:
#             pass
#         elif "zscore" in self.feature2:
#             days = int(''.join([n for n in self.feature2.split("_")[1] if n.isdigit()]))
#             self.data["{}_{}zscore".format(self.name, days)] = (
#                 (self.data[self.name].rolling(1).mean() - self.data[self.name].rolling(days).mean()) / 
#                 self.data[self.name].rolling(days).std())
#         elif "sma" in self.feature2:
#             days = int(''.join([n for n in self.feature2.split("_")[1] if n.isdigit()]))
#             #if a spread then uses its feature name
#             if self.spread1 and self.spread2:
#                 self.data[self.feature2] = self.data[self.feature2].rolling(days).mean()
#             else:
#                 self.data["{}_{}sma".format(self.name, days)] = self.data[self.name].rolling(days).mean()
#         elif "ema" in self.feature2: 
#             days = int(''.join([n for n in self.feature2.split("_")[1] if n.isdigit()]))
#             self.data["{}_{}ema".format(self.name, days)] = self.data[self.name].ewm(span=days).mean()
    
#     #where to chage if diff numbers in rule logic (subtract the number then compare to 0?)
#     def logic(self):
#         diff = []
#         self.data = self.data.fillna(method="bfill")
#         dates = list(data.index.values)

#         if type(self.feature1) == int or type(self.feature1) == float:
#             series1 = [self.feature1] * data.shape[0]
#         else:
#             series1 = self.data[self.feature1].to_list()
        
#         if type(self.feature2) == int or type(self.feature2) == float:
#             series2 = [self.feature2] * data.shape[0]
#         else:
#             series2 = self.data[self.feature2].to_list()

#         #iff = [round(((series1[x] - series2[x])/series1[x]) , 2) if series1[x] != 0 else (series1[x] - series2[x])
#         #       for x in range(data.shape[0])]
#         diff = []
#         for x in range(data.shape[0]):
#             if series1[x] > series2[x]:
#                 diff.append(1)
#             elif series1[x] <series2[x]:
#                 diff.append(-1)
#             else: diff.append(0)

#         signal = []
#         for x in diff:
#             if x > 0:
#                 #signal.append(round(1/13, 2))
#                 signal.append(1)
#             elif x < 0:
#                 signal.append(-1)
#             else:
#                 signal.append(0)
        
#         signal_df = pd.DataFrame(list(map(list, zip(dates, signal))))
#         signal_df.rename(columns = {signal_df.columns[0]: "Date", signal_df.columns[1]: self.name}, inplace = True)
    
        
#         return signal_df
)         

In [47]:
class Rule:
    def __init__(self, name, bucket, feature1, feature2, data=data, spread1=None, spread2=None, over_num=None, under_num=None,compare=None):
        self.name = name
        self.bucket = bucket
        self.data = data
        self.feature1 = feature1
        self.feature2 = feature2
        self.spread1 = spread1
        self.spread2 = spread2
        self.over_num = over_num
        self.under_num = under_num
        
        #needs work
        self.compare=compare

    #add lag by certain days funtion as well?

    def create_feature(self, one=True):
        if one:
            feat = self.feature1
        else:
            feat = self.feature2

        if type(feat) == int or type(feat) == float:
            pass       
        elif "zscore" in feat:
            days = int(''.join([n for n in feat.split("_")[1] if n.isdigit()]))
            if self.spread1 and self.spread2:
                self.data[feat] = (
                    (self.data[feat].rolling(1).mean() - self.data[feat].rolling(days).mean()) / 
                    self.data[feat].rolling(days).std())
            else:
                self.data["{}_{}zscore".format(self.name, days)] = (
                    (self.data[self.name].rolling(1).mean() - self.data[self.name].rolling(days).mean()) / 
                    self.data[self.name].rolling(days).std())
        elif "sma" in feat:
            days = int(''.join([n for n in feat.split("_")[1] if n.isdigit()]))
            #if a spread then uses its feature name
            if (self.spread1 and self.spread2) or self.compare:
                self.data[feat] = self.data[feat].rolling(days).mean()
            else:
                self.data["{}_{}sma".format(self.name, days)] = self.data[self.name].rolling(days).mean()
        elif "ema" in feat: 
            days = int(''.join([n for n in feat.split("_")[1] if n.isdigit()]))
            self.data["{}_{}ema".format(self.name, days)] = self.data[self.name].ewm(span=days).mean()    

    
    def generate_features(self):
        # Check to see if ratio needs to be calculated
        if (self.spread1 and self.spread2) and self.compare:
            if isinstance(self.feature1, str) and "atio" in self.feature1:
                self.data[self.feature1] = self.data[self.spread1] / self.data[self.spread2]
                self.data[self.feature2] = self.data[self.spread2] / self.data[self.spread1]
            else:
                print("Ratio Parameters inputted incorrectly!")
                return
        # Check to see if spread needs to be calculated
        elif self.spread1 and self.spread2:
            if (isinstance(self.feature1, str) or isinstance(self.feature2, str)) and ("pread" in str(self.feature1) or "pread" in str(self.feature2)):
                self.data[self.feature1] = self.data[self.spread1] - self.data[self.spread2]
                self.data[self.feature2] = self.data[self.spread1] - self.data[self.spread2] #added
            else:
                print("Spread Parameters inputted incorrectly!")
                return
        
        self.create_feature() #create feature1
        self.create_feature(False) #create feature 2

    
    #where to chage if diff numbers in rule logic (subtract the number then compare to 0?)
    def logic(self):
        diff = []
        self.data = self.data.fillna(method="bfill")
        dates = list(data.index.values)

        if type(self.feature1) == int or type(self.feature1) == float:
            series1 = [self.feature1] * data.shape[0]
        else:
            series1 = self.data[self.feature1].to_list()
        
        if type(self.feature2) == int or type(self.feature2) == float:
            series2 = [self.feature2] * data.shape[0]
        else:
            series2 = self.data[self.feature2].to_list()

        #create signal
        signal = []
        for x in range(data.shape[0]):
            #takes care of problem with 2 different measure numbers
            if self.over_num and self.under_num:
                if type(self.feature1) == int or type(self.feature1) == float:
                    if self.over_num  > series2[x]:
                        signal.append(1)
                    elif self.under_num < series2[x]:
                        signal.append(-1)
                    else: signal.append(0)
                elif type(self.feature2) == int or type(self.feature2) == float:
                    if series1[x] > self.over_num :
                        signal.append(1)
                    elif series1[x] < self.under_num:
                        signal.append(-1)
                    else: signal.append(0)
            else:
                if series1[x] > series2[x]:
                    signal.append(1)
                elif series1[x] <series2[x]:
                    signal.append(-1)
                else: signal.append(0)
                
                
        signal_df = pd.DataFrame(list(map(list, zip(dates, signal))))
        signal_df.rename(columns = {signal_df.columns[0]: "Date", signal_df.columns[1]: self.name}, inplace = True)
    
        
        return signal_df
            

In [48]:
def rule_factory(rules):

    bucket_list = []
    signal_dfs = []
    for rule in rules: 
        rule.generate_features()
        bucket_list.append(rule.bucket)
    for rule in rules: signal_dfs.append(rule.logic())
    
    heatmap_df = signal_dfs[0]
    
    for signal in signal_dfs[1:]:
        heatmap_df[signal.columns[1]]= signal[signal.columns[1]].to_list()
    heatmap_df = heatmap_df.set_index("Date", drop=True)
    
    #print out which rules were generated
    if bucket_list[0] == bucket_list[-1]:
        bucket_name = bucket_list[0]
    else:
        bucket_name = "All Rules"
    print("{} Signals Generated".format(bucket_name))
    
    #not using this?
    total_signal = []
    for index, row in heatmap_df.iterrows():
        row_weight = 0
        for col in heatmap_df.columns:
            weight = (row[col] )
            row_weight += weight
        total_signal.append(row_weight)
  
    #heatmap_df["Total Economic Signal"] = total_signal
    return heatmap_df

In [49]:
def plot(df, y_title="Date (Daily)", x_title="Signal"):
    fig = px.imshow(df, labels=dict(x=x_title, y=y_title), color_continuous_scale='RdYlGn')
    fig.update_xaxes(side="top")
    fig.show()
    #fig.write_image("{}.png".format(x_title))

In [50]:
"""
if feature1 > feature2, overweight signal
    elif feature1 < feature2: underwieght signal
    else: neutral signal
"""

# Total Rules: 14
# Missing: FED Composite Average Manufacturing Survey
economic_rules = [
    
Rule(
    #overwieght if 3 mth avg > 0
    name = "Bloomberg Eco US Surprise Index",
    bucket= "Economic",
    feature1 = "Bloomberg Eco US Surprise Index_60sma",
    feature2 = 0,
),

Rule(
    #overweight if 8mth z-score > 2yr z-score
    name = "US Auto Sales",
    bucket= "Economic",
    feature1 = "US Auto Sales_160zscore",
    feature2 = "US Auto Sales_500zscore",
),

Rule(
    #overweight if 10-2 spread > 0 #lagged 24 mths?
    name = "10y-2y Yield Spread",
    bucket= "Economic",
    feature1 = "10y-2y Yield Spread",
    feature2 = 0,
    spread1 = "TSY Yield 10Y",
    spread2 = "TSY Yield 2Y"
),
    
Rule(
    #overweight if 2 week avg > 3 mth avg
    name = "Bloomberg Commodity Metal Index",
    bucket= "Economic",
    feature1 = "Bloomberg Commodity Metal Index_10sma",
    feature2 = "Bloomberg Commodity Metal Index_60sma",
), 
    
Rule(
    #overweight if spot < 5 yr
    name = "Bloomberg Recession Probability Index",
    bucket= "Economic",
    feature1 = "Bloomberg Recession Probability Index_750sma",
    feature2 = "Bloomberg Recession Probability Index"
),
    
Rule(
    #overweight if 1 week avg > 8 mth avg
    name = "Citi Economic Surprise Index Eurozone",
    bucket= "Economic",
    feature1 = "Citi Economic Surprise Index Eurozone_5sma",
    feature2 = "Citi Economic Surprise Index Eurozone_160sma",
),

Rule(
    #overweight if spot > 52 #lagged 3d?
    name = "ISM Non-Manufacturing PMI",
    bucket= "Economic",
    feature1 = "ISM Non-Manufacturing PMI" ,
    feature2 = 52,
),

Rule(
    #overweight if 14 mth z-score < -0.2, underweight if 14 mth z-score > 0.2 
    name = "LME Copper Futures",
    bucket= "Economic",
    feature1 = 0, 
    feature2 = "LME Copper Futures_290zscore",
    over_num = -0.2,
    under_num = 0.2
),

Rule(
    #overweight if 3 mth avg > -0.2 #20d lag?
    name = "MoM Adjusted Retail Sales",
    bucket= "Economic",
    feature1 = "MoM Adjusted Retail Sales_60sma",
    feature2 = -0.2,
),
    
Rule(
    #overweight if 2 mth avg > 5 mth avg #8d lag?
    name = "US Average Weekly Work Hours",
    bucket= "Economic",
    feature1 = "US Average Weekly Work Hours_40sma",
    feature2 = "US Average Weekly Work Hours_100sma",
),
    
Rule(
    #overweight if 2 mth avg < 8 mth avg
    name = "US Reserve Balances with Fed",
    bucket= "Economic",
    feature1 = "US Reserve Balances with Fed_160sma",
    feature2 = "US Reserve Balances with Fed_40sma",
),

Rule(
    #overweight if 2 week avg < 6 mth avg
    name = "US Jobless Claims",
    bucket= "Economic",
    feature1 = "US Jobless Claims_125sma",
    feature2 = "US Jobless Claims_10sma",
),

Rule(
    #overweight if 3 mth avg > 6 mth avg
    name = "BDI Baltic Exchange Dry Index",
    bucket= "Economic",
    feature1 = "BDI Baltic Exchange Dry Index_60sma",
    feature2 = "BDI Baltic Exchange Dry Index_125sma",
),

Rule(
    #overweight if 2 mth avg > 0
    name = "Citi Economic Surprise Index US",
    bucket= "Economic",
    feature1 = "Citi Economic Surprise Index US_40sma",
    feature2 = 0,
),

Rule(
    #overweight if 3 mth avg > 12 mth avg
    name = "Industrial Production MoM %change",
    bucket= "Economic",
    feature1 = "Industrial Production MoM %change_60sma",
    feature2 = "Industrial Production MoM %change_250sma",
),


Rule(
    #overweight if 3 mth avg > 24 mth avg
    name = "US Empire Manufactoring Survey",
    bucket= "Economic",
    feature1 = "US Empire Manufactoring Survey_60sma",
    feature2 = "US Empire Manufactoring Survey_500sma",
) 


]

In [51]:
valuation_rules = [

Rule(
    #overwieght if 3mth > 10mth
    name = "MXUS Index EV/EBITDA",
    bucket= "Valuation",
    feature1 = "MXUS Index EV/EBITDA_60sma",
    feature2 = "MXUS Index EV/EBITDA_200sma",
),
    
Rule(
    #overwieght if 9 mth z-score > 0.75, underweight if < -0.25 
    name="MXUS Index FCF Yield",
    bucket="Valuation",
    feature1="MXUS Index FCF Yield_180zscore",
    feature2=0, 
    over_num = 0.75,
    under_num = -0.25
),
    
Rule(
    #overwieght if 10 mth avg > 3 mth avg
    name="MXUS Index Total Debt/EV",
    bucket="Valuation",
    feature1="MXUS Index Total Debt/EV_200sma",
    feature2="MXUS Index Total Debt/EV_60sma"
),

Rule(
    #overwieght if 2 mth avg > 2 yr avg, #TTM?
    name="MXUS Index Price to Sales Ratio",
    bucket="Valuation",
    feature1="MXUS Index Price to Sales Ratio_40sma",
    feature2="MXUS Index Price to Sales Ratio_500sma"
),

Rule(
    #overwieght if 12 mth avg < 20
    name="MXUS Price to Free Cash Flow",
    bucket = "Valuation",
    feature1=20,
    feature2="MXUS Price to Free Cash Flow_250sma"
),

Rule(
    #overwieght if 3 mth avg > 12 mth avg
    name="US Total Market PB Ratio",
    bucket = "Valuation",
    feature1="US Total Market PB Ratio_60sma",
    feature2="US Total Market PB Ratio_250sma"
),
    
Rule(
    #overwieght if 6 mth avg > spot
    name="MXUS Index PE",
    bucket = "Valuation",
    feature1="MXUS Index PE_120sma",
    feature2="MXUS Index PE"
),

Rule(
    #overwieght if 4 mth avg > spot
    name="MXUS Index Debt to Equity",
    bucket = "Valuation",
    feature1="MXUS Index Debt to Equity_80sma",
    feature2="MXUS Index Debt to Equity"
) ,

Rule(
    #overwieght if 5 mth avg > 15 mth avg
    name="MXUS Index ROE",
    bucket = "Valuation",
    feature1="MXUS Index ROE_100sma",
    feature2="MXUS Index ROE_310sma"
),

Rule(
    #overwieght if spread > 0
    name = "MXUS Dividend Yield vs 10yr Treasury Yield ",
    bucket= "Valuation",
    feature1 = "MXUS Dividend Yield vs 10yr Treasury Yield Spread",
    feature2 = 0,
    spread1 = "MXUS Index Divdend Yield",
    spread2 = "TSY Yield 10Y"
)

# Rule(
#     #weirdly overweight all the time?
#     #overwieght if spread  2 week avg > 10mth avg
#     name = "MXUS Total Market Cap vs US GNP Ratio",
#     bucket= "Valuation",
#     feature1 = "MXUS Total Market Cap vs US GNP Ratio_10sma",
#     feature2 = "MXUS Total Market Cap vs US GNP Ratio_200sma",
#     spread1 = "MXUS Total Market Cap",
#     spread2 = "US GNP",
#     compare = True
# )   
    
# Rule( #need to add to input -growth rate, logic problem needs work
#     #overwieght if spread  8mth Z-score < -0.5 & 8 mth avg > 0
#     name = "MXUS PEG Ratio",
#     bucket= "Valuation",
#     feature1 = ,
#     feature2 = ,
#     spread1 = ,
#     spread2 = 
# )   
    
]

In [52]:
sentiment_rules = [

Rule(
    #overweight if 0 > 3yr z-score
    name="CBOE Total Put Call Ratio",
    bucket="Sentiment",
    feature1=0,
    feature2="CBOE Total Put Call Ratio_750zscore"
),

Rule(
    #overweight if -10 > 1 week avg, underweight if 1 week avg > 10
    name="McClellan Oscillator",
    bucket="Sentiment",
    feature1=0, #place holder for now
    feature2="McClellan Oscillator_7sma",
    over_num = -10,
    under_num = 10
),

Rule(
    #overweight if 4 mth avg > 12 mth avg
    name="Mini S&P 500 Total Net Futures Positioning",
    bucket="Sentiment",
    feature1= "Mini S&P 500 Total Net Futures Positioning_80sma",
    feature2="Mini S&P 500 Total Net Futures Positioning_250sma"
),

Rule(
    #overweight if 1 mth avg > 5yr avg
    name="Bloomberg United States Financial Conditions Index",
    bucket="Sentiment",
    feature1= "Bloomberg United States Financial Conditions Index_20sma",
    feature2="Bloomberg United States Financial Conditions Index_1250sma"
),

Rule(
    #overweight if -1 > 36 mth z-score, underweight if 36 mth z-score > 1
    name="Bofa Global Financial Stress Indicator",
    bucket="Sentiment",
    feature1= 0,
    feature2="Bofa Global Financial Stress Indicator_750zscore",
    over_num = -1,
    under_num = 1
),

Rule(
    #overweight if 1yr avg > 1 mth avg
    name="BofA Securities GFSI Market Risk",
    bucket="Sentiment",
    feature1= "BofA Securities GFSI Market Risk_250sma",
    feature2="BofA Securities GFSI Market Risk_20sma"
),

Rule(
    #overweight if 3yr avg > 1 mth avg
    name="Bofa Securities GFSI SKEW",
    bucket="Sentiment",
    feature1= "Bofa Securities GFSI SKEW_750sma",
    feature2="Bofa Securities GFSI SKEW_20sma"
),


Rule(
    #overweight if 1 yr avg > 1 mth avg
    name="GS US Financial Conditions Index",
    bucket="Sentiment",
    feature1= "GS US Financial Conditions Index_250sma",
    feature2="GS US Financial Conditions Index_20sma"
),

Rule(
    #overweight if 14 mth avg > 4 mth avg
    name="Kansas City US Financial Stress Index",
    bucket="Sentiment",
    feature1= "Kansas City US Financial Stress Index_280sma",
    feature2="Kansas City US Financial Stress Index_80sma"
),

Rule(
    #overweight if abs(spot) > 12 mth avg #need to check abs()
    name="University of Michigan Consumer Expectations",
    bucket="Sentiment",
    feature1= "University of Michigan Consumer Expectations",
    feature2="University of Michigan Consumer Expectations_250sma"
),

Rule(
    #overweight if abs(spot) > 12 mth avg #need to check abs()
    name="University of Michigan Consumer Sentiment",
    bucket="Sentiment",
    feature1= "University of Michigan Consumer Sentiment",
    feature2="University of Michigan Consumer Sentiment_250sma"
),

Rule(
    #overweight if abs(spot) > 10 mth avg #need to check abs()
    name="University of Michigan Current Economic",
    bucket="Sentiment",
    feature1= "University of Michigan Current Economic",
    feature2="University of Michigan Current Economic_200sma"
),

Rule(
    #overweight if 7 mth avg > 1
    name="AAII US Investor Sentiment Bull/Bear Ratio",
    bucket="Sentiment",
    feature1= "AAII US Investor Sentiment Bull/Bear Ratio_140sma",
    feature2=1,
    spread1 = "AAII US Investor Sentiment Bullish Readings",
    spread2 = "AAII US Investor Sentiment Bearish Readings",
    compare = True
),

Rule(
    #overweight if 8 week avg > 2 week avg
    name="BofA Securities GFSI Liquidity Risk",
    bucket="Sentiment",
    feature1= "BofA Securities GFSI Liquidity Risk_40sma",
    feature2= "BofA Securities GFSI Liquidity Risk_10sma"
),

Rule(
    #overweight if 36 mth z-score < -1, underweight if 1 < 36 mth z-score
    name="High Yield OAS vs Investment Grade OAS Spread",
    bucket="Sentiment",
    feature1= 0, 
    feature2= "High Yield OAS vs Investment Grade OAS Spread_750zscore",
    spread1= "High Yield OAS",
    spread2= "Investment Grade OAS",
    over_num = -1,
    under_num = 1
),

Rule(
    #overweight if 1 mth avg > 4 mth avg
    name="VIX Index",
    bucket="Sentiment",
    feature1= "VIX Index_20sma",
    feature2= "VIX Index_80sma"
),

Rule( #need to add to input
    #overweight if 50d avg > 20d avg
    name="Credit Sussie Fear Barometer",
    bucket="Sentiment",
    feature1= "Credit Sussie Fear Barometer_50sma",
    feature2= "Credit Sussie Fear Barometer_20sma"
)

# Rule(
#     #index discontinued Jan '22
#     #overweight if 2 mth avg > 16 mth avg, #3d lagg?
#     name="Bloomberg US Buying Climate Comfort",
#     bucket="Sentiment",
#     feature1= "Bloomberg US Buying Climate Comfort_40sma",
#     feature2= "Bloomberg US Buying Climate Comfort_320sma"
# ),

# Rule(
#     #index discontinued Jan '22
#     #overweight if 1 week avg > 8 mth avg
#     name="Bloomberg Consumer Comfort Index",
#     bucket="Sentiment",
#     feature1= "Bloomberg Consumer Comfort Index_5sma",
#     feature2="Bloomberg Consumer Comfort Index_160sma"
# )
     
]

In [53]:
technical_rules = [
Rule(
   #overweight if spread 50d avg > 10d avg
   name = "SPX Return & FTSE 20Y Return Spread",
   bucket= "Technical",
   feature1 = "SPX Return & FTSE 20Y Return Spread_50sma",
   feature2 = "SPX Return & FTSE 20Y Return Spread_10sma", 
   spread1 = "SPX Total Return",
   spread2 = "FTSE US Treasury 20+ Years Index"
),

Rule(
    #overweight if 6mth avg > 0
    name = "Stock Price Strength Spread",
    bucket= "Technical",
    feature1 = "Stock Price Strength Spread_120sma",
    feature2 = 0,
    spread1=  "Total Number of New 52 Week Highs", 
    spread2 = "Total Number of New 52 Week Lows"
),  

Rule( 
    #naming convention will need ot change
    #overweight if high 50d avg > low 50d avg
    name = "Stock Highs vs Low",
    bucket= "Technical",
    feature1 = "Total Number of New 52 Week Highs Ratio_50sma",
    feature2 = "Total Number of New 52 Week Lows Ratio_50sma",
    spread1 = "Total Number of New 52 Week Highs",
    spread2 = "Total Number of New 52 Week Lows",
    compare = True
),   

Rule(
    #overweight if 2mth avg > 6 mth avg
    name = "NSYE Volume",
    bucket= "Technical",
    feature1 = "NSYE Volume_40sma",
    feature2 = "NSYE Volume_120sma"
),
    
Rule(
    #overweight if 8 mth expo avg > hist avg #will need to look more into total avg
    name = "McClellan Volume Summation Index Hist",
    bucket= "Technical",
    feature1 = "McClellan Volume Summation Index Hist_160ema", 
    feature2 = "McClellan Volume Summation Index Hist_2500sma" #place hold for entire mean
),

Rule(
    #overweight if 1mth avg > 8yr median #will need to look more into median
    name = "McClellan Volume Summation Index",
    bucket= "Technical",
    feature1 = "McClellan Volume Summation Index_20sma",
    feature2 = "McClellan Volume Summation Index_2000sma" #need median
),

Rule(
    #overweight if 1 mth avg > 10 mth avg
    name = "SPX Momentum 1M vs 10M",
    bucket= "Technical",
    feature1 = "SPX Momentum 1M vs 10M_20sma",
    feature2 = "SPX Momentum 1M vs 10M_200sma"
),

Rule(
    #overweight if 1 mth expo avg > 8mth expo avg
    name = "SPX Momentum 1M vs 8M",
    bucket= "Technical",
    feature1 = "SPX Momentum 1M vs 8M_20ema", 
    feature2 = "SPX Momentum 1M vs 8M_160ema" 
),

Rule(
    #overweight if10d expo avg > 70d expo avg
    name = "SPX Momentum 10d vs 70d",
    bucket= "Technical",
    feature1 = "SPX Momentum 10d vs 70d_10ema", 
    feature2 = "SPX Momentum 10d vs 70d_70ema" 
),
    

Rule(
    #overweight if 50d expo avg > 150d expo avg
    name = "SPX Momentum 50d vs 150d",
    bucket= "Technical",
    feature1 = "SPX Momentum 50d vs 150d_50ema", 
    feature2 = "SPX Momentum 50d vs 150d_150ema" 
),


Rule(
    #overweight if 50d avg > 200d avg
    name = "SPX Momentum 50d vs 200d",
    bucket= "Technical",
    feature1 = "SPX Momentum 50d vs 200d_50sma",
    feature2 = "SPX Momentum 50d vs 200d_200sma"
),

  
Rule(
    #overweight if 1 mth avg > 8 mth avg
    name = "SPX Volume",
    bucket= "Technical",
    feature1 = "SPX Volume_20sma",
    feature2 = "SPX Volume_160sma"
),

Rule(
    #overweight if 2 week avg > 6 mth avg
    name = "SPX Forward CAPE Ratio",
    bucket= "Technical",
    feature1 = "SPX Forward CAPE Ratio_10sma",
    feature2 = "SPX Forward CAPE Ratio_120sma"
),

Rule(
    #overweight if 12 mth avg > 2 mth avg
    name = "SPX FWD Earning Yield vs 3M TSY Yield Spread",
    bucket= "Technical",
    feature1 = "SPX FWD Earning Yield vs 3M TSY Yield Spread_250sma",
    feature2 = "SPX FWD Earning Yield vs 3M TSY Yield Spread_40sma",
    spread1 = "SPX FWD Yield",
    spread2 = "TSY Yield 3M"
)

# Rule(
#     #logic problem
#     #overweight if 1 mth avg > 8 mth avg
#     name = "SPX Momentum vs Volume",
#     bucket= "Technical",
#     feature1 = "SPX Volume_20sma",
#     feature2 = "SPX Volume_160sma",
# )
    
]

In [54]:
all_rules = []

for rule in economic_rules: all_rules.append(rule)
for rule in valuation_rules: all_rules.append(rule)
for rule in sentiment_rules: all_rules.append(rule)
for rule in technical_rules: all_rules.append(rule)
 

In [55]:
heatmap_econ_df = rule_factory(economic_rules)
heatmap_val_df = rule_factory(valuation_rules)
heatmap_sentiment_df = rule_factory(sentiment_rules)
heatmap_technical_df = rule_factory(technical_rules)

heatmap_all_df = rule_factory(all_rules)

Economic Signals Generated
Valuation Signals Generated
Sentiment Signals Generated
Technical Signals Generated
All Rules Signals Generated


## Daily Heatmaps for Each Bucket

In [56]:
plot(heatmap_econ_df.T, x_title="US EQ Signals - Economic", y_title="Rule Name")

In [57]:
plot(heatmap_val_df.T, x_title="US EQ Signals - Valuation", y_title="Rule Name")

In [58]:
plot(heatmap_sentiment_df.T, x_title="US EQ Signals - Sentiment", y_title="Rule Name")

In [59]:
plot(heatmap_technical_df.T, x_title="US EQ Signals - Technical", y_title="Rule Name")

## WEEKLY

In [60]:
plot(heatmap_econ_df.resample('W', label='left').mean().T, y_title= "Rule Name", x_title="US EQ Signals Weekly - Economic")

In [61]:
plot(heatmap_val_df.resample('W', label='left').mean().T, y_title= "Rule Name", x_title="US EQ Signals Weekly - Valuation")

In [62]:
plot(heatmap_sentiment_df.resample('W', label='left').mean().T, y_title= "Rule Name", x_title="US EQ Signals Weekly - Sentiment")

In [63]:
plot(heatmap_technical_df.resample('W', label='left').mean().T, y_title= "Rule Name", x_title="US EQ Signals Weekly - Technical")

## Monthly

In [64]:
plot(heatmap_econ_df.resample('M', label='left').mean().T, y_title= "Date (Monthly)", x_title="US EQ Signals Monthly - Economic")

In [65]:
plot(heatmap_val_df.resample('M', label='left').mean().T, y_title= "Date (Monthly)", x_title="US EQ Signals Monthly - Valuation")

In [66]:
plot(heatmap_sentiment_df.resample('M', label='left').mean().T, y_title= "Date (Monthly)", x_title="US EQ Signals Monthly - Sentiment")

In [67]:
plot(heatmap_technical_df.resample('M', label='left').mean().T, y_title= "Date (Monthly)", x_title="US EQ Signals Monthly - Technical")

In [68]:
plot(heatmap_all_df.resample('W', label='left').mean().T, y_title= "Rule Type", x_title="US EQ Signals Weekly - All Rules")

# Signal Gen for ML

In [69]:
# Generate Raw Features from ALL heatmaps
heatmap_all_df.to_csv('useq_raw_features.csv')

In [70]:
# Aggregate Heatmap output as featues (1 feature for each bucket)
def calc_bucket_score(df_heatmap, name='Econ Bucket'):
    ''' Calc overall bucket score for the day for use as ML feature'''
    s_signal = df_heatmap.sum(axis=1)
    score = round(
        (s_signal / df_heatmap.shape[1]), 2
    )
    
    df_score = score.to_frame(name=name)
    return df_score

df_heatmap_dict = {
    'Economic_signal' : heatmap_econ_df,
    'Valuation_signal': heatmap_val_df,
    'Sentiment_signal' : heatmap_sentiment_df,
    'Technical_signal' : heatmap_technical_df,
    'All_signal' : heatmap_all_df
}

l_dfs = []

for bucket in df_heatmap_dict:
    name = bucket.split("_")[0]
    df_score = calc_bucket_score(df_heatmap_dict[bucket], name)
    l_dfs.append(df_score)

df_heatmap_signal = pd.concat(l_dfs, axis=1)
df_heatmap_signal.to_csv('useq_heatmap_features.csv')
df_heatmap_signal

,Economic,Valuation,Sentiment,Technical,All
Date,,,,,
2015-01-01,0.12,-0.4,0.59,-0.21,0.09
2015-01-02,0.12,-0.4,0.59,-0.21,0.09
2015-01-03,0.12,-0.4,0.59,-0.21,0.09
2015-01-04,0.12,-0.4,0.59,-0.21,0.09
2015-01-05,0.12,-0.2,0.59,-0.43,0.07
...,...,...,...,...,...
2022-08-20,-0.19,-0.7,-0.71,-0.14,-0.42
2022-08-21,-0.25,-0.7,-0.71,0.00,-0.40
2022-08-22,-0.19,-0.7,-0.65,0.00,-0.37


In [71]:
plot(df_heatmap_signal.resample('W', label='left').mean().T, y_title= "Feature", x_title="US EQ Signals Featues Monthly")

## Seperate Heatmap by Bucket

In [72]:

def seperate_raw_data(buckets):
    for i, bucket in enumerate(buckets):
        # HACK: order of inputs is HARDCODED
        name_dict = {
            0: 'economic',
            1: 'sentiment',
            2: 'valuation', 
            3: 'technical_rules'
        }
        l_dfs = []
        for rule in bucket:
            if rule.name in data.columns:
                l_dfs.append(data[rule.name].to_frame())

        df_heatmap = pd.concat(l_dfs, axis=1)
        df_heatmap.to_csv('{}_raw_data.csv'.format(name_dict[i]))
        print('Bucket: {}, Rules: {}, Saved: {}'.format(name_dict[i], len(bucket), df_heatmap.shape[1]))


# Run data seperation 
buckets = [
    economic_rules,
    sentiment_rules,
    valuation_rules, 
    technical_rules
]

seperate_raw_data(buckets)

Bucket: economic, Rules: 16, Saved: 16
Bucket: sentiment, Rules: 17, Saved: 15
Bucket: valuation, Rules: 10, Saved: 9
Bucket: technical_rules, Rules: 14, Saved: 10
